# **POLICY ENGINE**

Motor de reglas encargado de decir cuando solicitar etiqueta al usuario

**Librerías**

In [ ]:
import time
from dataclasses import dataclass, field
from collections import deque
from typing import Deque, Tuple, Dict, Optional

**Variables para definir las reglas**

In [ ]:
# --- UMBRALES ---
MIN_CONF = 0.7        # incertidumbre sostenida
N_CONSEC = 7            # cuántas ventanas seguidas < MIN_CONF

SWITCH_MIN_CONF = 0.80  # confianza mínima para contar al switch
SWITCH_N_CONSEC = 10    # cuántas ventanas consecutivas con la NUEVA pred_label

COOLDOWN_S = 90        # anti-spam general entre preguntas
KEEP_ALIVE_S = 150      # 5 min
MAX_ASKS_PER_H = 25     # presupuesto/hora


**Estado por sesión**

In [ ]:
@dataclass
class SessionState:
    last_label_ts: float = 0.0
    last_ask_ts: float = 0.0
    asks_in_hour: Deque[float] = field(default_factory=lambda: deque(maxlen=64))
    last_conf: Deque[float] = field(default_factory=lambda: deque(maxlen=N_CONSEC))

    # tracking de cambio de actividad
    last_pred_label: str | None = None
    mode_label: str | None = None             # <- Estado estable actual del modelo

     # Swicth: seguimiento del candidato de cambio
    change_label: str | None = None
    change_run: int = 0

     # Última etiqueta confirmada por el usuario
    last_user_label: str | None = None
    last_keepalive_ts: float = 0.0
# clave: (id_usuario, session_id)
STATE: Dict[Tuple[int, int], SessionState] = {}

**Funciones para el motor de reglas**

In [ ]:
def _key(uid: int, sid: int) -> Tuple[int, int]:
    return int(uid), int(sid)

def _now(ts: float | None = None) -> float:
    return float(ts if ts is not None else time.time())

def _prune_hour(st: SessionState, now: float) -> None:
    one_hour_ago = now - 3600.0
    while st.asks_in_hour and st.asks_in_hour[0] < one_hour_ago:
        st.asks_in_hour.popleft()

def record_conf(uid: int, sid: int, conf: float) -> SessionState:
    st = STATE.setdefault(_key(uid, sid), SessionState())
    st.last_conf.append(conf)
    return st

def record_pred(uid: int, sid: int, label: str, conf: float) -> SessionState:
    st = STATE.setdefault(_key(uid, sid), SessionState())

    if conf >= SWITCH_MIN_CONF:
        if st.mode_label is None:
            # Primera vez: Se asume este label como el "modo" estable
            st.mode_label = label
            st.change_label = None
            st.change_run = 0
        elif label == st.mode_label:
            # Sigue igual → sin candidato
            st.change_label = None
            st.change_run = 0
        else:
            # Candidato de cambio: mismo label nuevo consecutivo => aumenta racha
            if st.change_label == label:
                st.change_run += 1
            else:
                st.change_label = label
                st.change_run = 1

    st.last_pred_label = label
    return st

def should_ask(uid: int, sid: int, conf: float, pred_label: str | None = None,
               now_ts: float | None = None) -> tuple[bool, str]:
    now = _now(now_ts)
    st = record_conf(uid, sid, conf)
    # anti-spam global

    if now - st.last_ask_ts < COOLDOWN_S:
        return False, "cooldown"

    if pred_label is not None:
        st = record_pred(uid, sid, pred_label, conf)

    #Preguntas por hora
    _prune_hour(st, now)
    if len(st.asks_in_hour) >= MAX_ASKS_PER_H:
        return False, "budget"

    # 1) cambio real
    if st.change_run >= SWITCH_N_CONSEC:
        return True, "switch"

    # 2) baja confianza sostenida
    if len(st.last_conf) == N_CONSEC and all(c < MIN_CONF for c in st.last_conf):
        return True, "uncertainty"

    # 3) keep-alive
    if st.last_label_ts == 0.0 or (now - max(st.last_label_ts, st.last_ask_ts)) >= KEEP_ALIVE_S:
        return True, "keep_alive"


    return False, "ok"

def mark_asked(uid: int, sid: int, now_ts: float | None = None, reason: str | None = None) -> None:
    now = _now(now_ts)
    st = STATE.setdefault(_key(uid, sid), SessionState())
    st.last_ask_ts = now
    _prune_hour(st, now)
    st.asks_in_hour.append(now)

    if reason == "keep_alive":
        st.last_keepalive_ts = now  # <- anti-spam de keep-alive

    if reason == "switch":
        # Fijamos el nuevo modo al candidato y reseteamos racha
        if st.change_label:
            st.mode_label = st.change_label
        st.change_label = None
        st.change_run = 0

def mark_labeled(uid: int, sid: int, now_ts: float | None = None, label: str | None = None) -> None:
    st = STATE.setdefault(_key(uid, sid), SessionState())
    now = _now(now_ts)
    st.last_label_ts = now
    st.last_keepalive_ts = now       # <- evita keep_alive inmediato tras etiquetar
    if label:
        st.last_user_label = label
        st.mode_label = label        # alinear “modo estable” con la etiqueta del usuario
    st.change_label = None
    st.change_run = 0

def feed_only(uid: int, sid: int, conf: float | None, pred_label: str | None = None):
    """
    Actualiza el estado interno (colas de confianzas y tracking de cambio)
    SIN evaluar cooldown/presupuesto y SIN devolver 'ask'.
    Útil para alimentar la policy desde el batch /predict_pending.
    """
    st = STATE.setdefault(_key(uid, sid), SessionState())

    if conf is not None:
        try:
            st.last_conf.append(float(conf))
            # la deque ya tiene maxlen=N_CONSEC (ver dataclass)
        except Exception:
            pass

def no_prediction_stats(df: pd.DataFrame) -> pd.DataFrame:
    """Detecta ventanas donde NINGÚN modelo dio predicción (los tres = null)."""
    mask = df["pred_label"].isna() & df["actividad"].isna() & df["actividad_b"].isna()
    df["_no_pred"] = mask.astype(int)

    gcols = ["id_usuario","session_id"]
    stats = df.groupby(gcols)["_no_pred"].agg(
        total="count",
        no_pred="sum"
    ).reset_index()
    stats["pct_no_pred"] = 100 * stats["no_pred"] / stats["total"]
    return stats

    if pred_label is not None:
        # mismo tracking que en record_pred, pero sin condiciones de ask
        label = str(pred_label)
        if conf is not None and conf >= SWITCH_MIN_CONF:
            if st.mode_label is None:
                st.mode_label = label
                st.change_label = None
                st.change_run = 0
            elif label == st.mode_label:
                st.change_label = None
                st.change_run = 0
            else:
                if st.change_label == label:
                    st.change_run += 1
                else:
                    st.change_label = label
                    st.change_run = 1
        st.last_pred_label = label
